In [1]:
import copy
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import subprocess
import glob
import re
from time import gmtime, strftime, sleep
from scipy import interpolate as interp
from keras.applications import resnet50
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Conv2DTranspose
from keras.constraints import maxnorm
from keras.layers import Activation
from keras.layers.convolutional import Conv2D 
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import backend as K
if K.backend()=='tensorflow':
    K.set_image_dim_ordering("tf")
 
# Import Tensorflow with multiprocessing
import tensorflow as tf
import multiprocessing as mp
%matplotlib inline
mpl.rcParams['figure.figsize'] = (16.0, 8.0) 

/Applications/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape= (268, 182, 3))

/Applications/anaconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [3]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 268, 182, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 274, 188, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 134, 91, 64)  9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 134, 91, 64)  256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [4]:
x = model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(26, activation='sigmoid')(x)
genre_model = Model(inputs = model.input, outputs = predictions)

In [5]:
genre_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 268, 182, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 274, 188, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 134, 91, 64)  9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 134, 91, 64)  256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [6]:
# adam = Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=0.0000001, decay=0.0, amsgrad=False)
genre_model.compile(loss='binary_crossentropy', optimizer='sgd')

In [9]:
#
# In this section, we distribute the images across three directories, 
# one for training, one for validation and one for testing.
#
import pandas as pd
import shutil
#load in X and Y

data_dir = '/home/odysseas/Dropbox/Family/John/cnn_posters'
image_files = [f for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))]
print('Loaded ' + str(len(image_files)) + ' images.')

train_dir = os.path.join(data_dir, 'train')
if not os.path.exists(train_dir):
    os.mkdir(train_dir)
    class1_dir = os.path.join(train_dir, 'class1')
    class2_dir = os.path.join(train_dir, 'class2')
    os.mkdir(class1_dir)
    os.mkdir(class2_dir)
    for fname in [image_files[index] for index in range(0,25)]:
        src = os.path.join(data_dir, fname)
        dst = os.path.join(class1_dir, fname)
        shutil.copyfile(src, dst)
    for fname in [image_files[index] for index in range(25,50)]:
        src = os.path.join(data_dir, fname)
        dst = os.path.join(class2_dir, fname)
        shutil.copyfile(src, dst)

validation_dir = os.path.join(data_dir, 'validation')
if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)
    class1_dir = os.path.join(validation_dir, 'class1')
    class2_dir = os.path.join(validation_dir, 'class2')
    os.mkdir(class1_dir)
    os.mkdir(class2_dir)
    for fname in [image_files[index] for index in range(50,55)]:
        src = os.path.join(data_dir, fname)
        dst = os.path.join(class1_dir, fname)
        shutil.copyfile(src, dst)
    for fname in [image_files[index] for index in range(55,60)]:
        src = os.path.join(data_dir, fname)
        dst = os.path.join(class2_dir, fname)
        shutil.copyfile(src, dst)

test_dir = os.path.join(data_dir, 'test')
if not os.path.exists(test_dir):
    os.mkdir(test_dir)
    for fname in [image_files[index] for index in range(60,70)]:
        src = os.path.join(data_dir, fname)
        dst = os.path.join(test_dir, fname)
        shutil.copyfile(src, dst)


Loaded 70 images.


In [13]:
from keras.preprocessing.image import ImageDataGenerator
from IPython.display import display
from PIL import Image

print(train_dir)
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
    target_size=(268, 182),
    batch_size=10,
    class_mode='binary')
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

validation_generator = train_datagen.flow_from_directory(validation_dir,
    target_size=(268, 182),
    batch_size=10,
    class_mode='binary')

history = genre_model.fit_generator(train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50)

/home/odysseas/Dropbox/Family/John/cnn_posters/train
Found 50 images belonging to 2 classes.
data batch shape: (10, 268, 182, 3)
labels batch shape: (10,)
Found 10 images belonging to 2 classes.
Epoch 1/30


ValueError: Error when checking target: expected dense_1 to have shape (26,) but got array with shape (1,)